<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW2/blob/main/NLP_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/.shortcut-targets-by-id/1Ikn75GANDVFyb0klJe4-EBQJikt4_Ob_/NLP


##Extracting essential data



In [31]:
import locale
print(locale.getpreferredencoding())

# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

# print(locale.getpreferredencoding())

UTF-8


In [34]:
# !pip install -q hazm
# !pip install -q transformers
!pip install  dadmatools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
from __future__ import unicode_literals
import hazm

import json
import pickle
import itertools

from tqdm import tqdm

In [9]:
with open('Nlp HW2/extracted-phrases.json','r',encoding='utf-8') as file:
    data = json.load(file)

KeyboardInterrupt: ignored

##Main code

###Config

In [37]:
from dadmatools.models.normalizer import Normalizer
import dadmatools.pipeline.language as language

dadma_normalizer = Normalizer(
    full_cleaning=True,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=True,
    remove_html=False,
    remove_stop_word=True,
    replace_email_with="<EMAIL>",
    replace_number_with="<NUM>",
    replace_url_with="",
    replace_mobile_number_with=None,
    replace_emoji_with=None,
    replace_home_number_with=None,
)

dadma_normalizer.normalize("بیا ای عقل کل با من که بردابرد او بینی")


pips = 'lem' 
nlp = language.Pipeline(pips)

print(nlp.analyze_pipes(pretty=True))

UnicodeDecodeError: ignored

In [11]:
corpus = [hazm.Lemmatizer().lemmatize(word[0]) for word in tqdm(hazm.words_list())]
with open("corpus.txt","wb") as fle:
  pickle.dump(corpus,fle)


100%|██████████| 35717/35717 [00:00<00:00, 1407691.68it/s]


In [12]:
with open("corpus.txt","rb") as file:
  corpus = pickle.load(file)

In [16]:
args = {
    'corpus':corpus,
    'pipe':nlp,
    'normalizer':dadma_normalizer,
    'translator':None,
    'word_tokenizer':hazm.WordTokenizer(),
    'sent_tokenizer':hazm.SentenceTokenizer()
}

In [26]:
class foreign_word_detector():

  def __init__(
      self,
      args
      ):
    self.corpus = args['corpus']
    self.lang_pipe = args['pipe']
    self.normalizer = args['normalizer']
    self.translator = args['translator']
    self.word_tokenizer = args['word_tokenizer']
    self.sent_tokenizer = args['sent_tokenizer']

  def detect(self, text):
    sentenced = self.sent_tokenizer.tokenize(text)
    lemmed = [str(self.lang_pipe(sent)) for sent in sentenced]
    normalized = [self.normalizer.normalize(sent) for sent in lemmed]
    words_list = [self.word_tokenizer.tokenize(sent) for sent in normalized]
    words_list = list(itertools.chain(*words_list))

    output = dict()
    out = []
    for word in words_list:
      if word not in self.corpus:
        out.append(word)
        begin = text.find(word)
        output[word] = [begin,begin+len(word)]
    
    return out, output


In [27]:
def run():
  detector = foreign_word_detector(args)

  text = ""
  print("enter your sample text.\nenter an empty string if you want to end the process")
  while True : 
    text = input()
    if text == "": 
      break
    
    out, output = detector.detect(text)
    print(out)
    print(output)


In [28]:
run()

enter your sample text.
enter an empty string if you want to end the process
سیستم کامپیوترم خراب شده است.
['سیستم', 'کامپیوتر']
{'سیستم': [0, 5], 'کامپیوتر': [6, 14]}
امروز تسک خیلی هاردی داشتیم اما تو موفق شدی کانتریبیوشن خوبی داشته باشی؛ تنکس.
['تسک', 'کانتریبیوشن', 'باشی', '؛', 'تنکس']
{'تسک': [6, 9], 'کانتریبیوشن': [44, 55], 'باشی': [67, 71], '؛': [71, 72], 'تنکس': [73, 77]}
من و علی در گاردنمان برگر خوردیم.
['گاردن', 'خوردیم']
{'گاردن': [12, 17], 'خوردیم': [26, 32]}
او بسیار آدم کولی بود
[]
{}
او بسیار آدم کولی بود.
[]
{}
امروز تایم ندارم باشد برای فردا.
['تایم']
{'تایم': [6, 10]}



problem is: using stemmer, we get lots of errors; while using lemmatizer, pronouns are not ommited from the word.


one way to deal with it would be to apply stemming right after dealing with the word origin.

another way would be to find a way to work with the tokenizer introduced in the next cell, since it gives us specific features.